### Tilaaja
Adconsys on suomalainen yritys, joka tuottaa rakennuksien automaatiojärjestelmiä ja automaatiojärjestelmien hallintajärjestelmiä.

### Tavoite

Luoda ennustusmalli, jolla voidaan ennustaa ilmanvaihtokoneen lämmityspatterille lähtevän veden lämpötilaa muiden järjestelmästä saatujen arvojen perusteella.

Nykyisin lämpötila lasketaan valmiin käyrän mukaan, joka on laskettu hieman yläkanttiin, jotta lämmityspatterilta palaavan vedeen lämpötila ei vahingossakaan laske liian alhaiseksi.

Tarkoituksena on korvata käyrä ennustusmallilla. Lämmityspatterille tulevan veden määrää säätävä venttiili on PID-säädin, joka laskee, kuinka paljon venttiilin tulisi olla avoinna menoveden mittausarvon ja lasketun arvon perusteella.

Jatkossa ennustusmallia on tarkoituksena muokata niin, että se pystyy ennakoimaan säätöarvoja esimerkiksi sääennustuksen ja energiankulutuksen pohjalta.

![](https://gitlab.dclabra.fi/wiki/uploads/upload_ced789c89dabfcce4b8b48654224b89f.png)

### Projektisuunnitelma
Tarkemmat vaatimukset ja reunaehdot löytyvät [projektisuunnitelmasta](https://gitlab.dclabra.fi/jvartiainen/proj3-team-adconsys/-/blob/master/Dokumentit/Projekti_3.pdf).

![](https://gitlab.dclabra.fi/wiki/uploads/upload_c3f3367eae12108794942a277100f299.png)

## Data-aineisto

### Formaatti
Saimme tilaajalta 10 kpl .xml tiedostoa. Nämä muutettiin .csv -muotoon, jotta niitä on helpompi käsitellä Pandasilla.

![](https://gitlab.dclabra.fi/wiki/uploads/upload_2fed80313cd1205b87833e4cdd9046a1.png)

Formaatin muutoksen jälkeen dataa on tämän verran:


|Tiedosto|Koko (MiB)|Rivimääärä|Alku|Loppu|
|---|---|---|---|---|
|IV_TE3A_Meno_M|19.3|533,439|18-May-20 8:30 PM EEST|07-May-21 7:57 AM EEST|
|IV_TE3A_Menovesi_LaskAs|0.8|21,752|03-Apr-19 8:51 AM EEST|07-May-21 7:36 AM EEST|
|TK1_LTO_S|3.9|108,333|16-May-19 3:16 PM EEST|06-May-21 4:03 AM EEST|
|TK1_TE1_2_PatterinPaluuvesi_M|3.8|106,585|04-Feb-20 10:40 PM EET|07-May-21 6:14 AM EEST|
|TK1_TE1_3_Tulo_M|2.1|59,388|17-Dec-19 10:06 AM EET|06-May-21 9:59 PM EEST|
|TK1_TE1_3_Tuloilma_LaskettuAs|0.24|6,576|03-Apr-19 8:51 AM EEST|03-May-21 4:39 PM EEST|
|TK1_TE1_4_Poisto_M|0.19|5,021|03-Apr-19 8:51 AM EEST|07-May-21 3:59 AM EEST|
|TK1_TF1_PE1_1_LaskAs|0.06|1,547|03-Apr-19 8:51 AM EEST|07-May-21 6:00 AM EEST|
|TK1_TV1_1_Lammitys_S|3.5|97,625|23-Jan-20 9:28 PM EET|07-May-21 6:23 AM EEST|
|Ulkolampotila|1.1|31,521|03-Apr-19 8:51 AM EEST|07-May-21 7:36 AM EES|

### Yhdistäminen

Jokaisessa tiedostossa on dataa eri aikaväleillä. Tiedostot täytyy saada yhdistettyä yhteen DataFrameen, koska useassa tiedostossa on dataa, joka täytyy siivota.

Virheellinen data täytyy saada poistettua jokaisesta tiedostosta samaan aikaan, sillä jos esimerkiksi ilmanvaihtokone on sammuksissa, ei minkään tiedoston data ole käytettävää.

Yhdistäminen tehdään timestamppien avulla. Yhdistämistä varten on timestampit muutettava datetime -muotoon, jolloin ajasta korjaantuu kesä-ja talviaika pois, ja se muuttuu 24h -muotoon.

Jokainen tiedoston timestampin tulee lopulliseen dataframeen. Rivien arvot haetaan edellisen arvon perusteella.

![](https://gitlab.dclabra.fi/wiki/uploads/upload_3e95bb5629d7fbe4536da3787a860f4c.png)

![](https://gitlab.dclabra.fi/wiki/uploads/upload_652cc790ad7042255e8fc6417ca0da0e.png)

Poistetaan duplikaatit ja NaN-arvot.

<!-- ![](https://gitlab.dclabra.fi/wiki/uploads/upload_27e0ac92dd1b6783af0601dd14f599d3.png) -->

![](https://gitlab.dclabra.fi/wiki/uploads/upload_1fd2aa0dc229c3e665f986a72fb72d31.png)

### Siivoaminen ja muokkaaminen

Datassa ennustuksen kannalta tärkeitä sarakkeita ovat value-sarakkeet, joiden kuvaajat näyttävät tältä:

![](https://gitlab.dclabra.fi/wiki/uploads/upload_298507780f6001df640b0254df56158a.png)

Dataa poistettiin Adconsysiltä saadun tiedon pohjalta seuraavasti:

#### Lämpötila ja säätöarvot
Lämpötilaa mittaavat sensorit tallentavat uuden arvon aina, kun lämpötilassa tapahtuu 0.5 asteen muutos. Säätöarvoja mittaavat sensorit tallentavat uuden arvon aina, kun säädössä tapahtuu 1% muutos.

Huomasimme, että järjestelmä on kuitenkin melko hidas, ja jos muutosta tapahtuu nopeasti, saataa kahden rivin välinen muutos olla edellä mainittuja arvoja huomattavasti suurempi. Päätimme, että emme erikseen lähde poistamaan dataa kähden rivin välisen muutoksen perusteella, koska melko helposti lähdetään poistamaan kunnollista dataa.

#### Status
Status -sarakeessa olevat arvot kertovat tallennetun mittaustuloksen "tilan". Fault- ja Down-arvot tarkoittavat, että systeemissä on jokin virhe, eikä mittaustulokset ole luotettavia. Nämä arvot voidaan suoraan poistaa. Overridden-arvot ovat käsin säädettyjä arvoja. Mittaus- ja säätödataa sisältävistä tiedostoista nämä voidaan poistaa.

#### TrendFlags
TrendFlags-sarakkeen arvoissa ei ole mitään tarpeellista tietoa.

#### LTO:n arvo < 100
Tarkoittaa, että lämmöntalteenottoa ei hyödynnetä täysin. Tällöin venttiilin TV1.1 on oltava suljettuna.

#### 1LaskAs == 0
Kone on kiinni. Automaatio pakottaa venttiilin täysin auki, eli säätöä ei tarvitse tehdä. 

#### Outlierit & Duplikaatit
Koneen sulkeitumisen johdosta dataan on muodostunut muutamia piikkejä. Nämä piikit poistetaan, sillä koneen sulkeituminenkin poistetaan datasta.
Myös muista value-sarakkeista poistetaan selkeät outlierit.

#### Normalisointi
Lopuksi muut sarakkeet paitsi ennustettettava sarake normalisoidaan.

![](https://gitlab.dclabra.fi/wiki/uploads/upload_8ab2b8a1ed3a2f6e7aef81e0494a04b7.png)

Jäljellä olevien rivien määrä, kun dataa on poistettu operaatioiden jälkeen.

![](https://gitlab.dclabra.fi/wiki/uploads/upload_81e450155d5c94931a45e4417b5fb5d5.png)

### Datan jakaminen

![](https://gitlab.dclabra.fi/wiki/uploads/upload_36ef02686e8f4af716f3737a84e92646.png)

Siivottu data jaetaan treeni-, testi- ja validointisetteihin.
Kokonaisdataa on 642011 riviä. Näistä 407,175 riviä jää treenisettiin, 117,269 riviä testisettiin ja 117,567 validointisettiin.
- Treeni 63.4%
- Testi 18.3%
- Validointi 18.3%

Testisettiin erotettiin kaikki '2021-02-15 11:21:00' jälkeen oleva data.
Loppu data jaettiin treeni- ja validointisetteihin siten, että joka viides päivä erotettiin validointisettiin.

Kuvaajasta nähdään miten data on jakautunut eri setteihin. On tärkeää, että jokaisessa datasetissä on dataa sekä viikoltä, että viikonlopulta, koska viikonloppuisin koulu on kiinni ja se vaikuttaa ilmanvaihtokoneen säätöihin.


![](https://gitlab.dclabra.fi/wiki/uploads/upload_bd2ef4050015ef146b8021916f17acc2.png)

Lopuksi timestamp poistetaan kaikista dataseteistä. Jokaisesta datasetistä luodaan oma piirrematriisi ja "IV_TE3A_Menovesi_LaskAs" -sarake erotetaan tavoitevektoriksi.


![](https://gitlab.dclabra.fi/wiki/uploads/upload_702dcb702ebf8152357edbe7684f65b8.png)

## Arkkitehtuurisuunnitelma

Eri tiedostoja ja datasettejä on useita ja ne kaikki on tallennettu gitlabiin. Gitlabin tarkempi kansiorakenne, sekä projektissa käytettävien kirjastojen versiot löytyvät [täältä](https://gitlab.dclabra.fi/jvartiainen/proj3-team-adconsys/-/blob/master/Dokumentit/Arkkitehtuurisuunnitelma_uusin.pdf).

## Testatut koneoppimismenetelmät

Testasimme projektissa vain luokitteluun käytettävää neuroverkkoa ja regressio-ongelmiin käytettävää verkkoa.

#### Luokitteluverkko

|Optimizer|Criterion|Activation|IP L|HL 2|HL 3|output|Accuracy|Epochs|Max +virhe| Max -virhe|Keskihajonta|mallit|
|---|---|---|---|---|---|---|---|---|---|---|---|---|
|```Adam(lr=0.001)```   |```CrossEntropyLoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128,67), LogSoftmax```|0.619  |3|5|-3|0.667|clas_CEL_3e.pth|
|```Adam(lr=0.001)```   |```CrossEntropyLoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128,67), LogSoftmax```|0.719  |5|15|-3|0.630|clas_CEL_5e.pth|
|```Adam(lr=0.001)```   |```MultiMarginLoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128,67), LogSoftmax```|0.691  |3|5|-3|0.670|clas_MM_3e.pth|
|```Adam(lr=0.001)```   |```MultiMarginLoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128,67), LogSoftmax```|0.710  |5|3|-25|0.600|clas_MM_5e.pth|
|```Adam(lr=0.001)```   |```MultiMarginLoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128,67), LogSoftmax```|0.746  |7|3|-2|0.594|clas_7e.pth|

Verkon hyvyyttä ei pysty pelkällä tarkkuudella määrittämään, sillä vaikka suurin osa ennustettavista lämpötiloista olisi oikein, saattaa loput olla hyvinkin kaukana tavoitearvosta.

Hyvyyttä voidaan mitata usealla eri tavalla, mutta parhaiten sitä voidaan arvoida kuvaajalla.

![](https://gitlab.dclabra.fi/wiki/uploads/upload_7e0eed18963c69fa79189072e1a321a8.png)

Luokitteluverkot olivat meidän ensimmäinen testi, ja niillä saatiin yllättävän hyviä tuloksia todella nopeasti.

### Regressioverkko

|Optimizer|Criterion|Activation|IP L|HL 2|HL 3|HL 4|HL 5|output|Accuracy|Epochs|Max +virhe| Max -virhe|Keskihajonta|mallit|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|```Adam(lr=0.001)```   |```MSELoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128, 64)```|```Linear(64, 32)```|```Linear(32, 1)```|-  |3|0.790|-1.685|0.222|reg_3epoch.pth|
|```Adam(lr=0.001)```   |```MSELoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128, 64)```|```Linear(64, 32)```|```Linear(32, 1)```|-  |5|1.622|-1.011|0.218|reg_5_e.pth|
|```Adam(lr=0.001)```   |```MSELoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128, 64)```|```Linear(64, 32)```|```Linear(32, 1)```|-  |5|1.066|-1.674|0.212|reg_5epoch.pth|
|```Adam(lr=0.001)```   |```MSELoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128, 64)```|```Linear(64, 32)```|```Linear(32, 1)```|-  |7|0.607|-1.267|0.212|reg_7epoch.pth|
|```Adam(lr=0.001)```   |```MSELoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128, 64)```|```Linear(64, 32)```|```Linear(32, 1)```|-  |10|1.622|-1.137|0.239|n_red10e.pth|
|```Adam(lr=0.001)```   |```MSELoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128, 64)```|```Linear(64, 32)```|```Linear(32, 1)```|-  |10|1.119|-0.896|0.205|reg_10epoch.pth|
|```Adam(lr=0.001)```   |```MSELoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128, 64)```|```Linear(64, 32)```|```Linear(32, 1)```|-  |15|0.871|-1.080|0.218|reg_15epoch.pth|

![](https://gitlab.dclabra.fi/wiki/uploads/upload_d77edf5e091aa6eae696d5848aff010d.png)

## Tulosten pohdinta

- Regressioverkolla saadaan huomattavasti parempia tuloksia verrattuna luokitteluverkkoon.
- Ulkolämpötila korreloi lähes 100% ennustettavan menoveden kanssa, joten mallin on melko helppo oppia sen ennustaminen.
- Testidata on vuodenajalta, jolla mallia ei ole ollenkaan opetettu. Silti se ennustaa hyvin tarkasti. Näistä syistä mallin ennustuksiin voidaan luottaa.
- Malli on opetettu yhden rakennuksen datalla, joten se oletettavasti toimii vain sille rakennukselle. 

- Alkuperäinen tavoite oli luoda malli, jolla voidaan minimoida lämmityskustannuksia ennustamalla, millä saatöarvoilla saavutetaan riittävä menoveden lämpötila siten, että tavoitearvot saavutetaan.
    - Saimme toiseksi viimeisellä viikolla kaavan, jolla voidaan laskea energiankulutus.
    - Pidemmälle olisi voinut päästä, jos olisi tuntemusta säätöjärjestelmistä.
- Saimme kuitenkin luotua mallin, jolla voidaan laskea mikä menoveden tavoitearvon tulisi olla.

## Mahdolliset jatkokehityskohteet

- Datan siivoamisessa ja esikäsittelyssä tulisi tuntea ilmanvaihtosysteemi paremmin, sekä mitkä kaikki arvot todella ovat oleellisia lopullisen ennustuksen kannalta.
- Ennustusmallin tarkkuutta voi varmasti vielä parantaa säätämällä sen parametrejä.
    - Olisi myös ollut mielenkiintoista testata miten malli ennustaa jostakin toisesta rakennuksesta kerätyllä datalla.
- Energian kulutuksen minimointi.
- Mallin käyttöönotto. 
    - Ennustumallille .py tiedosto, joka voidaan ottaa käyttöön toimeksiantajan järjestelmässä.
    - Ehdot milloin mallin tarvitsee ennustaa. LTO:n hyötyprosentti ja venttiilin säätöarvot.
- Kesäajalle voisi testata omaa mallia, sillä koulu on kiinni, eikä lämmityksestä tarvi silloin kovin paljoa välittää.